<a href="https://colab.research.google.com/github/gghatano/BKB/blob/master/%E6%B0%97%E3%81%AE%E5%88%A9%E3%81%84%E3%81%9F%E6%93%AC%E4%BC%BC%E3%83%87%E3%83%BC%E3%82%BF%E3%82%92%E7%94%9F%E6%88%90%E3%81%99%E3%82%8B%EF%BC%9ASDV%E3%81%AE%E3%83%81%E3%83%A5%E3%83%BC%E3%83%88%E3%83%AA%E3%82%A2%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# "気の利いた" 擬似データを作成したい

## 実データの取得は大変
- データ分析やシステム開発のために、実データが必要になることは多々あります。
- ただ、実データは限られた環境でしか扱えない/利用に際して多くの稟議を通さなくちゃいけない/etc...など、データ取得までは時間と両力がかかります。(注意して使わないといけないものなので、当然です)
- 解決策として、データの見た目や統計量などの「保存して欲しい性質」を残しつつ、実データとは一定以上異なる「高度な擬似データ」を利用できる場合があります。

## 擬似データ生成手法と課題
- 擬似データ生成を行う仕組みはいくつか提案されています。見た目や統計的な性質を保存しつつ、元データとは(適当な基準により)異なる、と言えるようなデータを、機械的に生成することができる手法が多いです。
  - (いくつか既存手法を書いておく)
- ただ、得られた擬似データに、元データのルールを上手に反映するのは、難しいです。
  - 例えば、「年齢」「職歴」という列があったら、「5歳」「アルバイト」みたいな状態はありえない...というのが一般的かなと思います。
  - こういうあり得ないデータを、あまり生成したくありません。

## 今回やること：SDVを使って、制約を反映した擬似データを生成する
- 常識や業務的なルールを反映しやすそうな仕組みを持つ、擬似データ生成ライブラリ[SDV](https://github.com/sdv-dev/SDV)について、理解を深めてみます。
  - [チュートリアル](https://sdv.dev/SDV/user_guides/single_table/handling_constraints.html)を参考に、実際に動かしてみます。

# SDV-チュートリアルの内容確認

:
## SDVのインストール
- pipから導入できます

In [ ]:
! pip install sdv
from sdv.demo import load_tabular_demo
import pandas as pd


## 扱うデータ・ルールの確認
- SDVに含まれている、従業員データセットを確認してみます。
  - 仮想の複数社に所属する従業員の方の、所属や給料、雇用形態などが含まれるダミーデータです。

In [ ]:
employees = load_tabular_demo()
print(employees.columns)
employees.head()

Index(['company', 'department', 'employee_id', 'age', 'age_when_joined',
       'years_in_the_company', 'salary', 'annual_bonus',
       'prior_years_experience', 'full_time', 'part_time', 'contractor'],
      dtype='object')


,company,department,employee_id,age,age_when_joined,years_in_the_company,salary,annual_bonus,prior_years_experience,full_time,part_time,contractor
0,Pear,Sales,1,41,38,3,97000.00,13000.00,1,1.0,0.0,0.0
1,Pear,Design,5,49,42,7,77172.68,13241.72,2,0.0,0.0,1.0
2,Glasses,AI,1,38,34,4,49500.00,21500.00,3,1.0,0.0,0.0
3,Glasses,Search Engine,7,45,41,4,52125.94,8804.94,2,0.0,0.0,1.0
4,Cheerper,BigData,6,47,40,7,47500.00,11500.00,3,0.0,1.0,0.0


## データのルールの確認
よく観察してデータの意味を考えると、いくつか制約が思いつきます。例えば...
- 一意性制約
  - company(会社)ごとに一意なemployee_id(社員ID)が振られるはずなので、「(company, employee_id)は一意」
- 不等式で表現できる制約 (定数との比較や他列との比較)
  - salary(月給？)の値を考えると、「$\text{salary} \geq 0$」
  - age(今の年齢)とage_when_joined(入社した時の年齢)の関係を考えると、「$\text{age} \geq \text{age_when_joined}$」
  - age年齢なので、従業員っぽい年齢(一旦16歳から80歳までとします)なはずなので、「$16\leq\text{age}\leq 80$」
- 等式で表現できる制約
  - 「$\text{age} - \text{age_when-joined} = \text{years_in_the_company}$」
- 値のデータ型に関する制約
  - salary列の表現を参考に、「$\text{salary}$は小数第2位で表す」
  - $\text{full_time}$, $\text{part_time}$, $\text{contractor}$は、どれかが1で他が0になっている(3種類の値が含まれる契約形態列をone-hot-encodingしたもの、という背景があるみたいです)：「$\text{full_time} + \text{part_time} + \text{contractor} = 1$,  $\text{full_time}, \text{part_time}, \text{contractor} \in \{0,1\}$」

などがあり、それぞれを反映した擬似データを作る...ための仕組みが用意されています。

(前処理や後処理で対応する方が良さそうなもの...もありますが、管理の手間の観点で、データのルールを全部擬似データ生成時に反映する、という戦略もアリな気がしています)





## Reject samplingについて
制約を反映する方法には2通りあって、
- 上手い変換を考える
  - $\text{age} \geq \text{age_when-joined}$ について、$\text{diff} = \text{age} - \text{age_when-joined}$の値だけ保存しておく
  - $\text{age_when-joined}$を生成して、$\text{diff}$を足してできる列を$\text{age}$とする 
- 制約を満たさないレコードを棄却して、生成し直す

## 制約の反映

### 一意性制約
- sdv.constraints.Uniqueを使います
  - 一意になる列の組を入力します
  

In [ ]:
from sdv.constraints import Unique
unique_employee_id_company_constraint = Unique(
    columns = ['company', 'employee_id']
)

## データの生成
- データ生成の手法には幾つかあり、大まかには「コピュラベースの手法」「深層学習ベースの手法」が用意されています。同じように使えるので、両方を触ってみます。

In [ ]:
## 制約の反映
from sdv.tabular import GaussianCopula
from sdv.tabular import CTGAN
constraints = [
               unique_employee_id_company_constraint
]
gc = GaussianCopula(constraints=constraints)
ct = CTGAN(constraints=constraints)

In [ ]:
## 学習・データ生成
gc.fit(employees)
ct.fit(employees)
sampled_gc = gc.sample(100)
sampled_ct = gc.sample(100)

/usr/local/lib/python3.7/dist-packages/scipy/stats/_continuous_distns.py:639: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/usr/local/lib/python3.7/dist-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/scipy/stats/_continuous_distns.py:5320: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
/usr/local/lib/python3.7/dist-packages/scipy/stats/_distn_infrastructure.py:2606: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/usr/local/lib/python3.7/dist-packages/copulas/univariate/truncated_gaussian.py:43: RuntimeWarning: invalid value encountered in double_scalars
  a = (self.min - loc) / scale
/usr/local/lib/python3.7/dist-packages/copulas/univariate/truncated_ga

In [ ]:
## 内容確認
sampled_gc.head()

,company,department,employee_id,age,age_when_joined,years_in_the_company,salary,annual_bonus,prior_years_experience,full_time,part_time,contractor
0,Cheerper,BigData,27,31,25,5,37570.88,20875.04,2,0.0,1.0,0.0
1,Cheerper,AI,9,40,33,4,96523.89,14440.50,2,1.0,0.0,0.0
2,Glasses,Search Engine,1,49,47,3,42449.92,18159.33,1,0.0,1.0,0.0
3,Pear,Design,21,36,27,8,44104.33,9730.32,3,0.0,0.0,1.0
4,Glasses,AI,38,38,36,1,66113.15,19152.20,2,1.0,0.0,0.0


In [ ]:
sampled_ct.head()

,company,department,employee_id,age,age_when_joined,years_in_the_company,salary,annual_bonus,prior_years_experience,full_time,part_time,contractor
0,Pear,Sales,1,48,42,4,71737.32,10587.92,2,1.0,0.0,0.0
1,Glasses,AI,57,49,45,4,67896.10,11020.15,4,0.0,0.0,1.0
2,Glasses,AI,1,37,32,3,51383.26,24500.00,3,1.0,0.0,0.0
3,Glasses,Support,18,49,46,4,100189.79,6800.41,2,0.0,0.0,0.0
4,Glasses,AI,29,37,36,1,88555.73,15394.55,4,1.0,0.0,1.0
